In [ ]:
# default_exp wholesale_price_processor

# Wholesale Price Analytics

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import pandas as pd
from fastcore.script import *

In [ ]:
# hide
!pip install pdftotext

  Using cached pdftotext-2.2.0.tar.gz (113 kB)
  Created wheel for pdftotext: filename=pdftotext-2.2.0-cp38-cp38-linux_x86_64.whl size=66673 sha256=d640568fa73c8d27b8112662fd49be36e8042df009e95018af443a47bbf2ace1
  Stored in directory: /home/rasta/.cache/pip/wheels/f9/c4/d5/093622266ee915c03f8cd5165cc119cc2e356d548d8fd3c1b0
Successfully built pdftotext


In [ ]:
#export
import PyPDF2
import pdftotext
import pandas as pd
import os

In [ ]:
#export
def create_header(pdftotext_lines:list):
    header_index = 4
    header = pdftotext_lines[header_index].split()
    temp = header[3:]
    temp[0:] = [header[0]]
    temp.insert(1, "Variety source")
    temp[2:]  = [header[4]+ " " + header[5]]
    temp[3:]  = [header[6]+ " " + header[7]]
    temp[4:]  = [header[8]+ " " + header[9] + " " + header[10]]
    return temp

In [ ]:
#export
def remove_empty_items(lst:list):
    return list(filter(lambda item: len(item.lstrip()) > 0,  lst) )  


In [ ]:
#export
def remove_surrounding_spaces_from_items(lst:list):
    return list(map(lambda item: item.lstrip(),  lst) )  

In [ ]:
#export
def get_data(pdftotext_lines:list):
    data = [] 
    lines = pdftotext_lines[9:]
    for line in lines:
        if  "Prepared" in line:
            continue
        if "Commodity" in line:
            continue
        if "Ministry" in line:
            continue
        if "Kingston" in line:
            continue
        if "Commerce" in line:
            continue
        if "Week" in line:
            continue
        if "Fisheries" in line: 
            continue
        d = remove_empty_items(line.split('   '))
        d = remove_surrounding_spaces_from_items(d)
        data.append(d)
        
    
    return data


In [ ]:
#export
def write_to_csv(data, report_date,  output_directory):
   
    output_dir = f"{output_directory}/{report_date}"
    try:
        os.mkdir(output_dir) 
    except OSError as error: 
        print(f"{output_dir} already exists,not recreating")      
 
    df = pd.DataFrame(data)
    out = f"{output_dir}/kingston_supermarket_prices_{report_date}.csv"
    print(f"writing data to: {out}")
    df.to_csv(out,sep="|" ,index=False, header=False)  
   

In [ ]:
#export  
def get_report_date(pdftotext_lines:list):
    date = pdftotext_lines[2]
    date = date.replace("Week","")
    date = date.replace("Ending","").replace("\n","").lower().lstrip().replace(" ","_").replace(",","")
    return date

In [ ]:
#export
def process_wholesale_price_data(input_path:str, output_directory = "."):
    with open(input_path, 'rb') as file:
        pdf_text = pdftotext.PDF(file)
        pdf_text_lines = ("\n\n".join(pdf_text).splitlines())
        pdftotext_lines = [ln for ln in  pdf_text_lines if ln]
        report_date = get_report_date(pdftotext_lines)
        print(report_date.lstrip())
        header =  create_header(pdftotext_lines)
        data = get_data(pdftotext_lines)
        data[:0] =[header]
        write_to_csv(data, report_date, output_directory)
    


In [ ]:
input_path="../data/raw/wholesale_09.04.2021.pdf"
output_directory = "../data/output"

run(input_path,output_directory)        

In [ ]:
!head ../data/output/september_04_2021/kingston_supermarket_prices_september_04_2021.csv

Commodity|Variety source|Lowest Price|Highest Price|Most Frequent Price
Cabbage (Red)|Local|-|-|-
Callaloo|Shredded|$286|$308|$297
Callaloo|Whole|$154|$154|$154
Cantaloupe|Imported|-|-|-
Cantaloupe|Local|$198|$264|$227.33
Carrot|Local|$264|$330|$330
Cauliflower|Imported|-|-|-
Cauliflower|Local|$620|$660|$620
Coco|Local|$330|$440|$330


In [ ]:
#export
@call_parse
def main(input_path:Param("The urban municipal prices file path", str),
         output_folder:Param("where the converted file should be written", str),
      ):
    "Clean up the input file and generates individual parish farm gate prices in the specified output folder"
    print(f"input path:{input_path}")
    run(input_path, output_folder)
